In [1]:
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col

In [2]:
%fs ls dbfs:/FileStore/tables/

path,name,size
dbfs:/FileStore/tables/HIST_PAINEL_COVIDBR_04jul2020.xlsx,HIST_PAINEL_COVIDBR_04jul2020.xlsx,28924600
dbfs:/FileStore/tables/indicadoressegurancapublicauffev20-1.xlsx,indicadoressegurancapublicauffev20-1.xlsx,499712
dbfs:/FileStore/tables/indicadoressegurancapublicauffev20-2.xlsx,indicadoressegurancapublicauffev20-2.xlsx,499712
dbfs:/FileStore/tables/indicadoressegurancapublicauffev20-3.xlsx,indicadoressegurancapublicauffev20-3.xlsx,499712
dbfs:/FileStore/tables/indicadoressegurancapublicauffev20.xlsx,indicadoressegurancapublicauffev20.xlsx,499712
dbfs:/FileStore/tables/pageviews_by_second_example-1.tsv,pageviews_by_second_example-1.tsv,3663
dbfs:/FileStore/tables/pageviews_by_second_example-2.tsv,pageviews_by_second_example-2.tsv,3663
dbfs:/FileStore/tables/pageviews_by_second_example-3.tsv,pageviews_by_second_example-3.tsv,3663
dbfs:/FileStore/tables/pageviews_by_second_example-4.tsv,pageviews_by_second_example-4.tsv,3663
dbfs:/FileStore/tables/pageviews_by_second_example.tsv,pageviews_by_second_example.tsv,3663


In [3]:
# # usando option("inferSchema", "true") não se faz necessária a declaração do schema
# schema = StructType([StructField("UF",StringType(), True),
#                      StructField("TP_CRIME",StringType(), True),
#                      StructField("ANO",StringType(), True),
#                      StructField("MES",StringType(), True),
#                      StructField("QTD_VITIMAS",StringType(), True)
#                     ])

# sparkDF = spark.read.format("com.databricks.spark.csv").option("header", "true").schema(schema).option("delimiter", ";").option("encoding", "iso-8859-1").load('dbfs:/FileStore/tables/seguranca_publicafev20-1.csv')\
# .withColumn("QTD_VITIMAS", col("QTD_VITIMAS").cast(IntegerType()))

# display(sparkDF)

In [4]:
sparkDF = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").option("delimiter", ";").option("encoding", "iso-8859-1").load('dbfs:/FileStore/tables/seguranca_publicafev20-1.csv')
display(sparkDF)

UF,Tipo Crime,Ano,Mês,Vítimas
Acre,Homicídio doloso,2015,janeiro,13
Acre,Homicídio doloso,2015,fevereiro,10
Acre,Homicídio doloso,2015,março,16
Acre,Homicídio doloso,2015,abril,22
Acre,Homicídio doloso,2015,maio,15
Acre,Homicídio doloso,2015,junho,14
Acre,Homicídio doloso,2015,julho,14
Acre,Homicídio doloso,2015,agosto,13
Acre,Homicídio doloso,2015,setembro,18
Acre,Homicídio doloso,2015,outubro,15


In [5]:
sp = sparkDF.filter(sparkDF['UF']== 'São Paulo')
display(sp)

UF,Tipo Crime,Ano,Mês,Vítimas
São Paulo,Homicídio doloso,2016,dezembro,362
São Paulo,Homicídio doloso,2017,janeiro,310
São Paulo,Homicídio doloso,2017,fevereiro,312
São Paulo,Homicídio doloso,2017,março,320
São Paulo,Homicídio doloso,2017,abril,303
São Paulo,Homicídio doloso,2017,maio,273
São Paulo,Homicídio doloso,2017,junho,258
São Paulo,Homicídio doloso,2017,julho,283
São Paulo,Homicídio doloso,2017,agosto,265
São Paulo,Homicídio doloso,2017,setembro,272


In [6]:
HD_SP = sp.filter(sp['Tipo Crime']== 'Homicídio doloso')
display(HD_SP)

UF,Tipo Crime,Ano,Mês,Vítimas
São Paulo,Homicídio doloso,2016,dezembro,362
São Paulo,Homicídio doloso,2017,janeiro,310
São Paulo,Homicídio doloso,2017,fevereiro,312
São Paulo,Homicídio doloso,2017,março,320
São Paulo,Homicídio doloso,2017,abril,303
São Paulo,Homicídio doloso,2017,maio,273
São Paulo,Homicídio doloso,2017,junho,258
São Paulo,Homicídio doloso,2017,julho,283
São Paulo,Homicídio doloso,2017,agosto,265
São Paulo,Homicídio doloso,2017,setembro,272


In [7]:
from pyspark.sql.functions import lit
vitimas_HD_sp1 = HD_SP.groupBy('Ano').sum('Vítimas').select('Ano',col('sum(Vítimas)').alias('numero_Vítimas'))
vitimas_HD_sp = vitimas_HD_sp1.withColumn('UF', lit('São Paulo')).withColumn('Tipo Crime', lit('Homicídio doloso'))
display(vitimas_HD_sp)

Ano,numero_Vítimas,UF,Tipo Crime
2018,3106,São Paulo,Homicídio doloso
2019,2906,São Paulo,Homicídio doloso
2020,495,São Paulo,Homicídio doloso
2016,362,São Paulo,Homicídio doloso
2017,3503,São Paulo,Homicídio doloso


In [8]:
rj = sparkDF.filter(sparkDF['UF']== 'Rio de Janeiro')
HD_RJ = rj.filter(rj['Tipo Crime']== 'Homicídio doloso')
vitimas_HD_RJ1 = HD_RJ.groupBy('Ano').sum('Vítimas').select('Ano',col('sum(Vítimas)').alias('numero_Vítimas'))
vitimas_HD_RJ = vitimas_HD_RJ1.withColumn('UF', lit('Rio de Janeiro')).withColumn('Tipo Crime', lit('Homicídio doloso'))
display(vitimas_HD_RJ)

Ano,numero_Vítimas,UF,Tipo Crime
2018,4950,Rio de Janeiro,Homicídio doloso
2015,4200,Rio de Janeiro,Homicídio doloso
2019,3999,Rio de Janeiro,Homicídio doloso
2020,672,Rio de Janeiro,Homicídio doloso
2016,5042,Rio de Janeiro,Homicídio doloso
2017,5346,Rio de Janeiro,Homicídio doloso


In [9]:
Homicidios_RJ_SP = vitimas_HD_sp.union(vitimas_HD_RJ)
Homicidios_RJ_SP=Homicidios_RJ_SP.filter(Homicidios_RJ_SP['Ano'] != '2015')
display(Homicidios_RJ_SP.sort('Ano'))

Ano,numero_Vítimas,UF,Tipo Crime
2016,362,São Paulo,Homicídio doloso
2016,5042,Rio de Janeiro,Homicídio doloso
2017,3503,São Paulo,Homicídio doloso
2017,5346,Rio de Janeiro,Homicídio doloso
2018,3106,São Paulo,Homicídio doloso
2018,4950,Rio de Janeiro,Homicídio doloso
2019,2906,São Paulo,Homicídio doloso
2019,3999,Rio de Janeiro,Homicídio doloso
2020,495,São Paulo,Homicídio doloso
2020,672,Rio de Janeiro,Homicídio doloso
